# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather = pd.read_csv('WeatherPy.csv')
weather.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Lushnjë,AL,1612311686,40.9419,19.7050,44.60,93,3,3.44
1,1,Laguna,US,1612311657,38.4210,-121.4238,62.01,77,40,6.91
2,2,Upernavik,GL,1612311686,72.7868,-56.1549,-6.74,87,100,4.12
3,3,Haines Junction,CA,1612311687,60.7522,-137.5108,-7.60,54,73,0.25
4,4,Kapaa,US,1612311447,22.0752,-159.3190,78.80,65,40,13.80


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#variables
lat_long = weather[['Lat', 'Lng']]
humidity = weather['Humidity']
lat_long

,Lat,Lng
0,40.9419,19.7050
1,38.4210,-121.4238
2,72.7868,-56.1549
3,60.7522,-137.5108
4,22.0752,-159.3190
...,...,...
544,-20.1883,32.6236
545,42.0551,59.3788
546,9.6697,122.4256
547,56.9064,118.2631


In [4]:
#creating heatmap layer taken from class assignment
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
humid_layer = gmaps.heatmap_layer(lat_long, weights=humidity,
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(humid_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
dream_vacay_df = weather.loc[(weather["Max Temp"] >= 75) & (weather["Wind Speed"] <= 10) & (weather["Cloudiness"] <=3)] 
dream_vacay_df = dream_vacay_df.dropna()
dream_vacay_df

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
69,69,Aloleng,PH,1612311704,16.1307,119.7824,76.78,69,1,9.62
213,213,Salalah,OM,1612311741,17.0151,54.0924,75.20,12,0,6.91
252,252,Guarapari,BR,1612311752,-20.6667,-40.4975,79.00,78,0,9.22
323,323,Umm Kaddadah,SD,1612311770,13.6017,26.6876,75.31,20,0,8.81
375,375,Boueni,YT,1612311784,-12.9025,45.0761,82.40,78,0,6.91
406,406,Ocós,GT,1612311792,14.5094,-92.1933,82.40,74,1,4.61
407,407,Igbo-Ora,NG,1612311792,7.4333,3.2833,79.45,73,0,3.94
457,457,Pitimbu,BR,1612311804,-7.4706,-34.8086,78.80,83,0,6.91
461,461,Lázaro Cárdenas,MX,1612311805,17.9583,-102.2000,80.08,72,0,9.57
477,477,El Centro,US,1612311627,32.7920,-115.5630,78.80,33,1,3.44


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
#storing varialbe
hotel_df = dream_vacay_df

In [30]:
#creating column
hotel_df['Hotel Name'] = ""
hotel_df.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
69,69,Aloleng,PH,1612311704,16.1307,119.7824,76.78,69,1,9.62,
213,213,Salalah,OM,1612311741,17.0151,54.0924,75.20,12,0,6.91,
252,252,Guarapari,BR,1612311752,-20.6667,-40.4975,79.00,78,0,9.22,
323,323,Umm Kaddadah,SD,1612311770,13.6017,26.6876,75.31,20,0,8.81,
375,375,Boueni,YT,1612311784,-12.9025,45.0761,82.40,78,0,6.91,


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [32]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig

NameError: name 'heat_layer' is not defined